In [1]:
library(dplyr)
library(parallel)
library(stringr)

# hubname='CU'
# continent='na'
# lakename='PP'
# PT_key_file='SWOTCalVal_PP_KEY_20230515_20230627.csv'
# utm_zone=14 #WM= 10

# hubname='CU'
# continent='na'
# lakename='PW'
# PT_key_file= c('SWOTCalVal_PW_KEY_20230314_20230430.csv','SWOTCalVal_PW_KEY_20230424_20230719.csv') #WM
# #PT_key_file= c('SWOTCalVal_PW_KEY_20230424_20230719_TR.csv') #WM
# utm_zone=10 #WM= 10

# hubname='UNC'
# continent='na'
# lakename='SL'
# PT_key_file= 'SWOTCalVal_SL_KEY_20230710_20231018.csv' #WM
# utm_zone=11 #WM= 10

hubname='UNC'
continent='na'
lakename='YF'
PT_key_file=c('SWOTCalVal_YF_KEY_20230521_20230923.csv',
              'SWOTCalVal_YF_KEY_20230603_20230722.csv') #WM
utm_zone=6 #WM= 10

setwd(paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/'))
# raw_GNSS_drift_data_directory=paste0('From Andy/',hubname,'_netCDFs/')
# QA_QC_drift_output_directory= paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/Munged_lake_gnss/')
# QA_QC_PT_output_directory= paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/Munged_lake_PT/')
# flagged_drift_output_directory=paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/flagged_lake_drifts/')
# flagged_PT_output_directory=paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/flagged_lake_PT/')
# raw_PT_data_directory= paste0('/nas/cee-water/cjgleason/calval/xml_scripts/',hubname,'/Munged/')

raw_GNSS_drift_data_directory=paste0('From Andy/',hubname,'_netCDFs/')
QA_QC_drift_output_directory= paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/Munged_lake_gnss_TR/')
QA_QC_PT_output_directory= paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/Munged_lake_PT_TR/')
flagged_drift_output_directory=paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/flagged_lake_gnss_TR/')
flagged_PT_output_directory=paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/flagged_lake_PT_TR/')
raw_PT_data_directory= paste0('/nas/cee-water/cjgleason/calval/xml_scripts/',hubname,'/Munged/')


read_keys=function(keyfile){
    
    # Read in key file and check column names and NODE ID for precision lost with scientific notation #
  this_key= read.csv(keyfile,stringsAsFactors=FALSE, na.strings = c("","NA","na","NaN", " "))%>%
    mutate(keyid=keyfile)%>%
    mutate(pt_serial=as.integer(PT_Serial))
    
    }
    
### Key file get and check ###    
master_key= do.call(rbind,lapply(PT_key_file,read_keys))


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# ncol(read.csv('SWOTCalVal_PW_KEY_20230424_20230719_nodate.csv'))
# ncol(read.csv('SWOTCalVal_PW_KEY_20230314_20230430.csv'))

[1] 27

[1] 27

In [5]:
#create dataframes from drifts---------------------------------------------------------
#pull filename before the .csv
source('/nas/cee-water/cjgleason/calval_toolbox/R code/create_lake_GNSS_dataframe.R')
raw_GNSS=sub( "\\..*","", list.files(raw_GNSS_drift_data_directory,recursive=TRUE))
raw_GNSS_lake=which(!is.na(do.call(rbind,lapply(raw_GNSS,str_match,lakename))))
raw_GNSS=raw_GNSS[raw_GNSS_lake]

 parse_gnss=function(gnss_file){
    splitter= strsplit(gnss_file,'_')
    first_two_dates=paste(splitter[[1]][6],splitter[[1]][7],sep="_")
    
    #the standard date format has an 'nT' character in the middle of thedates
    third_date=as.POSIXct(paste0(substring(splitter[[1]][8],1,8),
                                 substring(splitter[[1]][8],10,15) ),
                          format='%Y%m%d%H%M%S')
    # tp=splitter[[1]][[12]]
    return(data.frame(cbind(keystring=paste(first_two_dates),third_date)))
  }
  
  
  
  gnss_index=do.call(rbind,lapply(raw_GNSS,parse_gnss))%>%
    #this is implicit, so when we filter things we'll mess up the order. 
    #add an index favlue
    mutate(gnss_file_index=1:nrow(.))%>%
    group_by(keystring)%>%
    filter(third_date==max(third_date))
  
 
  #remove old files
  raw_GNSS=raw_GNSS[gnss_index$gnss_file_index]


#pull filename before the second _
QA_QC_drifts=sub( "\\..*","",list.files(QA_QC_drift_output_directory))
flagged_drifts=sub("\\..*","",list.files(flagged_drift_output_directory))
#what raw drift data have not been munged
unmunged_drifts=setdiff(raw_GNSS,c(flagged_drifts,QA_QC_drifts))
# # print(raw_GNSS)
# print(unmunged_drifts)
# bonk

for (i in 1:length(unmunged_drifts)){
create_lake_gnss_dataframe(unmunged_drifts[i],
                  gnss_drift_data_directory=raw_GNSS_drift_data_directory,
                  output_directory=QA_QC_drift_output_directory)}

# cl=makeCluster(44)

#   dummy=parLapply(cl=cl,unmunged_drifts,create_lake_gnss_dataframe,
#                   gnss_drift_data_directory=raw_GNSS_drift_data_directory,
#                   output_directory=QA_QC_drift_output_directory)
# stopCluster(cl)



[1] "/nas/cee-water/cjgleason/calval/Processed data/UNC/Munged_lake_gnss_TR/SWOTCalVal_YF_GNSS_L2_Rec12_20230721T203305_20230721T215100_20230808T030555.csv"
[1] "/nas/cee-water/cjgleason/calval/Processed data/UNC/Munged_lake_gnss_TR/SWOTCalVal_YF_GNSS_L2_Rec12_20230721T223040_20230721T233720_20230808T033603.csv"
[1] "/nas/cee-water/cjgleason/calval/Processed data/UNC/Munged_lake_gnss_TR/SWOTCalVal_YF_GNSS_L2_Rec12_20230722T001958_20230722T014622_20230808T035556.csv"
[1] "/nas/cee-water/cjgleason/calval/Processed data/UNC/Munged_lake_gnss_TR/SWOTCalVal_YF_GNSS_L2_Rec12_20230722T022631_20230722T032631_20230808T041930.csv"
[1] "/nas/cee-water/cjgleason/calval/Processed data/UNC/Munged_lake_gnss_TR/SWOTCalVal_YF_GNSS_L2_Rec12_20230722T160900_20230722T171123_20230808T044039.csv"
[1] "/nas/cee-water/cjgleason/calval/Processed data/UNC/Munged_lake_gnss_TR/SWOTCalVal_YF_GNSS_L2_Rec12_20230722T174800_20230722T190205_20230808T050159.csv"
[1] "/nas/cee-water/cjgleason/calval/Processed data/UNC/Mu

In [2]:
#correct PTs to the GNSS from the key to the lake
source('/nas/cee-water/cjgleason/calval_toolbox/R code/lake_PT.r')

gnss_sd_thresh=0.15 # 15cm how much variance do you want in the GNSS data when it is within the distance threshold?
offset_sd_thresh=0.10 #m, so 10cm. the the PT apparantly shift by more than a cm?
change_thresh_15_min=0.05#m- does it change more than 5cm in 15 minutes? that is a discontinuity in offset
dist_thresh=200
time_thresh=15*60
dry_threshold=0.10

#get all the PTs in the data directory
munged_files= list.files(raw_PT_data_directory,recursive= TRUE)
PT_files=munged_files[grepl(paste0("_", lakename, "_PT_L1"), munged_files)]
# PT_index=which(!is.na(do.call(rbind,lapply(munged_files,str_match,'PT_L1'))))
# PT_files=munged_files[PT_index]

csv_index=which(!is.na(do.call(rbind,lapply(PT_files,str_match,'.csv'))))
csv_PT_files=PT_files[csv_index]

# correct_lake_index=which(!is.na(do.call(rbind,lapply(csv_PT_files,str_match,lakename))))
# csv_PT_files=csv_PT_files[correct_lake_index]
 #there are 'bad' files within the PT xml folders that we do not want to bring in, so filter by version date.
  # positions 8 and 9 are the start and end of the data
  # position 10 is the repro date- we want the most recent repro
  parse_pt=function(pt_file){
    # sans_csv = sub('\\.csv$', '',pt_file)
    splitter= strsplit(pt_file,'_')
    first_two_dates=paste(splitter[[1]][8],splitter[[1]][9],sep="_")
    #the standard date format has an 'nT' character in the middle of thedates
    third_date=as.POSIXct(paste0(substring(splitter[[1]][10],1,8),
                                 substring(splitter[[1]][10],10,15) ),
                          format='%Y%m%d%H%M%S')
    # tp=splitter[[1]][[12]]
    return(data.frame(cbind(keystring=paste0(splitter[[1]][7],'_',first_two_dates),third_date)))
  }
  
  pt_index=do.call(rbind,lapply(csv_PT_files,parse_pt))%>%
    #this is implicit, so when we filter things we'll mess up the order. 
    #add an index favlue
    mutate(pt_file_index=1:nrow(.))%>%
    group_by(keystring)%>%
    filter(third_date==max(third_date))
  
  #remove old files
  csv_PT_files=csv_PT_files[pt_index$pt_file_index]

#pull filename before the .csv
raw_PT=sub("\\..*","",csv_PT_files)
raw_PT=sub(".*/","",raw_PT)
#pull filenam\\..*e before the second _
QA_QC_PTs=sub("\\..*","",list.files(QA_QC_PT_output_directory))

#what raw PT data have not been munged
    #need to preserve the full path in csv_PT_files!
unmunged_PTs=setdiff(csv_PT_files,QA_QC_PTs)
# print(unmunged_PTs)

 for (i in 1:length(unmunged_PTs)){
test=lake_PT(unmunged_PTs[i],
             master_key=master_key,
             dist_thresh=NULL,
             time_thresh=time_thresh,
             pt_data_directory=raw_PT_data_directory,
             gnss_drift_data_directory=QA_QC_drift_output_directory,
             QA_QC_PT_output_directory=QA_QC_PT_output_directory, 
             flagged_pt_output_directory=flagged_PT_output_directory,
             gnss_sd_thresh=gnss_sd_thresh,
             offset_sd_thresh=offset_sd_thresh,
             change_thresh_15_min=change_thresh_15_min,
             dry_threshold=dry_threshold)
 }


    # cl=makeCluster(44,type='FORK')
  
#   dummy=parLapply(cl, unmunged_PTs,lake_PT,
#              pt_key_file=PT_key_file,
#              pt_data_directory=raw_PT_data_directory,
#              dist_thresh=dist_thresh,
#              time_thresh=time_thresh,
#              gnss_drift_data_directory=QA_QC_drift_output_directory,
#              QA_QC_pt_output_directory=QA_QC_PT_output_directory,
#              flagged_pt_output_directory=flagged_PT_output_directory,
#              gnss_sd_thresh=GNSS_sd_thresh,
#              offset_sd_thresh=offset_sd_thresh,
#              change_thresh_15_min=change_thresh_15_min)
  
#   stopCluster(cl)





Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




[1] "/nas/cee-water/cjgleason/calval/xml_scripts/UNC/Munged/Munged__20231106/SWOTCalVal_YF_PT_L1_PT225_20230524T150000_20230828T211500_20231106T145020.csv"
[1] 2153226
[1] "Date is good"
[1] "Date is in correct format"


`summarise()` has grouped output by 'occupy_id'. You can override using the
`.groups` argument.


[1] "SWOTCalVal_YF_PT_L1_PT225_20230524T150000_20230828T211500_20231106T145020_SWOTCalVal_YF_KEY_20230521_20230923.csv"
[1] "this file passed all checks"
[1] "/nas/cee-water/cjgleason/calval/xml_scripts/UNC/Munged/Munged__20231106/SWOTCalVal_YF_PT_L1_PT226_20230524T150000_20230828T210000_20231106T145020.csv"
[1] 2153241
[1] "Date is good"
[1] "Date is in correct format"


`summarise()` has grouped output by 'occupy_id'. You can override using the
`.groups` argument.


...

[1] "SWOTCalVal_YF_PT_L1_PT235_20230524T150000_20230828T213000_20240110T171125_SWOTCalVal_YF_KEY_20230521_20230923.csv"
[1] "this file passed all checks"
[1] "/nas/cee-water/cjgleason/calval/xml_scripts/UNC/Munged/Munged__20240110/SWOTCalVal_YF_PT_L1_PT242_20230524T150000_20230722T210000_20240110T171126.csv"
[1] 2119362
[1] "Date is good"


`summarise()` has grouped output by 'occupy_id'. You can override using the
`.groups` argument.


[1] "SWOTCalVal_YF_PT_L1_PT242_20230524T150000_20230722T210000_20240110T171126_SWOTCalVal_YF_KEY_20230603_20230722.csv"
[1] "this file passed all checks"
[1] "/nas/cee-water/cjgleason/calval/xml_scripts/UNC/Munged/Munged__20240329/SWOTCalVal_YF_PT_L1_PT001_20230419T200000_20231012T160000_20240329T174535.csv"
[1] 2153089
[1] "Date is good"
[1] "/nas/cee-water/cjgleason/calval/xml_scripts/UNC/Munged/Munged__20240401/SWOTCalVal_YF_PT_L1_PT224_20230524T150000_20230828T204500_20240401T152147.csv"
[1] 2153097
[1] "Date is good"


`summarise()` has grouped output by 'occupy_id'. You can override using the
`.groups` argument.


[1] "SWOTCalVal_YF_PT_L1_PT224_20230524T150000_20230828T204500_20240401T152147_SWOTCalVal_YF_KEY_20230521_20230923.csv"
[1] "this file passed all checks"


In [55]:
as.integer(substring(read.table("/nas/cee-water/cjgleason/calval/xml_scripts/CU/Munged/Munged__20240117/SWOTCalVal_PW_PT_L1_PT131_20230322T003000_20230426T234500_20240117T181415.csv",nrow=1)$V1,1,7))
  

[1] 2153229